In [1]:
import arcpy
import os

# 定义坐标 (左下角、右下角、右上角、左上角)
coordinates = [
    [92, 32],   # 左下角
    [109, 32],  # 右下角
    [109, 43],  # 右上角
    [92, 43],   # 左上角
    [92, 32]    # 回到左下角
]
# 创建空间参考（例如：WGS 1984）
spatial_ref = arcpy.SpatialReference(4326)  # WGS 1984

# 创建多边形，并指定空间参考
polygon = arcpy.Polygon(arcpy.Array([arcpy.Point(*coord) for coord in coordinates]), spatial_ref)

# 获取当前路径
current_path = r"C:\Users\r\Desktop\arcen"

# 指定 gansu_loc 文件夹路径
loc_folder = os.path.join(current_path, "arcen", "gansu_loc")

# 如果 gansu_loc 文件夹不存在，创建它
if not os.path.exists(loc_folder):
    os.makedirs(loc_folder)

# 创建一个新的要素类并保存多边形到 gansu_loc 文件夹
gdb_path = os.path.join(loc_folder, "gansu_loc.gdb")
arcpy.management.CreateFileGDB(loc_folder, "gansu_loc")  # 创建 gansu_loc.gdb
arcpy.management.CreateFeatureclass(gdb_path, "gansu_loc", "POLYGON", spatial_reference=spatial_ref)  # 创建 gansu_loc 要素类，并指定空间参考

# 插入多边形到要素类
with arcpy.da.InsertCursor(os.path.join(gdb_path, "gansu_loc"), ["SHAPE@"]) as cursor:
    cursor.insertRow([polygon])

# 设置符号系统
aprx_path = r"C:\Users\r\Desktop\arcen\arcen\arcen.aprx"  # 指定 .aprx 文件的路径
aprx = arcpy.mp.ArcGISProject(aprx_path)  # 加载指定的项目文件
map_obj = aprx.listMaps()[0]  # 获取第一个地图
lyr = map_obj.addDataFromPath(os.path.join(gdb_path, "gansu_loc"))  # 添加数据到地图
sym = lyr.symbology  # 获取符号系统
sym.renderer.symbol.color = {'RGB': [255, 0, 0, 100]}  # 设置红色
lyr.symbology = sym  # 应用符号系统

# Optional: Save the project if you want to keep the changes
aprx.save()